In [10]:
import sys
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import numpy as np

In [11]:
# Adiciona o diretório raiz do projeto ao sys.path
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_path not in sys.path:
    sys.path.append(project_path)

In [13]:
from src.modeling import split_data, train_model, evaluate_model, save_metrics, predicted_rent

In [16]:
df = pd.read_csv('../data/processed/dataset_imoveis_sp_clean.csv')

X_train, X_test, y_train, y_test = split_data(df, target='rent')  # Ajuste o nome da coluna 'rent', se necessário

# Treinar o modelo
model = train_model(X_train, y_train, model_type='ridge')  # Usando regularização Ridge

# Avaliar o modelo
metrics = evaluate_model(model, X_test, y_test)
print("Métricas do modelo:", metrics)

# Salvar o modelo treinado
joblib.dump(model, '../outputs/linear_regression_model.pkl')

# Caminho para salvar as métricas
metrics_path = '../src/metrics.txt'  # Certifique-se de que o diretório 'src' tenha permissões para escrita

# Salvar as métricas no arquivo de texto
save_metrics(metrics, metrics_path)
print(f"Métricas salvas em {metrics_path}")


Métricas do modelo: {'MAE': 1101.9690016775614, 'RMSE': np.float64(1640.832359474773), 'R2': 0.5704544972983068}
Métricas salvas em ../src/metrics.txt


In [15]:
model = joblib.load('../outputs/linear_regression_model.pkl')  # Ajuste o caminho se necessário

new_data = pd.DataFrame({
    'area': [200, 80, 120],
    'bedrooms': [2, 2, 3],
    'garage': [3, 1, 2],
    'zone_Centro': [1, 0, 0],
    'zone_Leste': [0, 0, 0],
    'zone_Norte': [0, 1, 0],
    'zone_Oeste': [0, 0, 1],
    'zone_Sul': [0, 0, 0],
    'type_Apartamento': [1, 0, 0],
    'type_Casa': [0, 1, 0],
    'type_Casa em condomínio': [0, 0, 1],
    'type_Studio e kitnet': [0, 0, 0],
})

predictions = predicted_rent(model, new_data)

# Arredondar as previsões para 2 casas decimais
rounded_predictions = np.round(predictions, 2)

# Exibir as previsões arredondadas
print("Previsões dos aluguéis (arredondadas):")
for i, prediction in enumerate(rounded_predictions):
    print(f"Propriedade {i+1}: R$ {prediction:.2f}")


Previsões dos aluguéis (arredondadas):
Propriedade 1: R$ 7688.90
Propriedade 2: R$ 1888.81
Propriedade 3: R$ 5639.20
